In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

import tensorflow as tf
from tensorflow.keras import layers, losses, Input
from tensorflow.keras.models import Model

2022-05-12 16:43:20.301531: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Links:

- https://blog.keras.io/building-autoencoders-in-keras.html
- https://www.tensorflow.org/tutorials/generative/autoencoder


In [2]:
pretrain_features = pd.read_csv('handout/pretrain_features.csv')
pretrain_labels = pd.read_csv('handout/pretrain_labels.csv')
train_features = pd.read_csv('handout/train_features.csv')
train_labels = pd.read_csv('handout/train_labels.csv')
test_features = pd.read_csv('handout/test_features.csv')

## Cleaning data

In [3]:
for c in pretrain_features.columns[2:]:
    if len(pretrain_features[c].unique()) < 2:
        pretrain_features = pretrain_features.drop(c, axis=1)
        train_features = train_features.drop(c, axis=1)
        test_features = test_features.drop(c, axis=1)

## Autoencoder

In [4]:
class Autoencoder(Model):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim   
        self.encoder = tf.keras.Sequential([
            Input(shape=(input_dim,)),
            layers.Dense(256, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(latent_dim)
        ])
        self.decoder = tf.keras.Sequential([
            Input(shape=(latent_dim,)),
            layers.Dense(128, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(input_dim)
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [5]:
X_pretrain_train, X_pretrain_val, y_pretrain_train, y_pretrain_val = train_test_split(
    pretrain_features[pretrain_features.columns[2:]], pretrain_labels[["lumo_energy"]], test_size=0.001, random_state=42
)

In [6]:
autoencoder = Autoencoder(X_pretrain_train.shape[1], latent_dim=32)
autoencoder.compile(optimizer='adam', loss=losses.MSE)

2022-05-12 16:44:41.805344: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-12 16:44:41.807547: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-12 16:44:41.807591: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-12 16:44:41.807642: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (t3ui01.psi.ch): /proc/driver/nvidia/version does not exist
2022-05-12 16:44:41.808268: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild Ten

In [7]:
history = autoencoder.fit(X_pretrain_train, X_pretrain_train, epochs=15, batch_size=32, validation_data=(X_pretrain_val, X_pretrain_val))

2022-05-12 16:44:59.464632: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 389210400 exceeds 10% of free system memory.
2022-05-12 16:45:00.145744: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 389210400 exceeds 10% of free system memory.
2022-05-12 16:45:00.612892: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-05-12 16:45:00.613465: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2294660000 Hz


Epoch 1/15
1561/1561 [==============================] - 5s 3ms/step - loss: 0.0220 - val_loss: 0.0141
Epoch 2/15
1561/1561 [==============================] - 4s 2ms/step - loss: 0.0133 - val_loss: 0.0123
Epoch 3/15
1561/1561 [==============================] - 4s 2ms/step - loss: 0.0117 - val_loss: 0.0114
Epoch 4/15
1561/1561 [==============================] - 4s 2ms/step - loss: 0.0108 - val_loss: 0.0107
Epoch 5/15
1561/1561 [==============================] - 4s 2ms/step - loss: 0.0102 - val_loss: 0.0102
Epoch 6/15
1561/1561 [==============================] - 4s 2ms/step - loss: 0.0097 - val_loss: 0.0098
Epoch 7/15
1561/1561 [==============================] - 4s 2ms/step - loss: 0.0094 - val_loss: 0.0096
Epoch 8/15
1561/1561 [==============================] - 4s 2ms/step - loss: 0.0091 - val_loss: 0.0095
Epoch 9/15
1561/1561 [==============================] - 4s 2ms/step - loss: 0.0089 - val_loss: 0.0093
Epoch 10/15
1561/1561 [==============================] - 4s 2ms/step - loss: 0.008

In [14]:
class ELumon(Model):
    def __init__(self, autoencoder):
        super(ELumon, self).__init__()
        self.encoder = autoencoder.encoder
        self.regressor = tf.keras.Sequential([
            Input(shape=self.encoder.output_shape[1:]),
            layers.BatchNormalization(),
            layers.Dense(16, activation='relu'),
            layers.Dense(8, activation='relu'),
        ])
        self.top = tf.keras.Sequential([
            Input(shape=self.regressor.output_shape[1:]),
            layers.Dense(1, activation='linear', kernel_regularizer="l2")
        ])
        # Set all layers to trainable
        for layer in self.encoder.layers:
            layer.trainable = True

    def call(self, x):
        encoded = self.encoder(x)
        reg = self.regressor(encoded)
        return self.top(reg)

elumon = ELumon(autoencoder)
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
elumon.compile(optimizer=opt, loss="mean_squared_error")

X_pretrain_train, X_pretrain_val, y_pretrain_train, y_pretrain_val = train_test_split(
    pretrain_features[pretrain_features.columns[2:]], pretrain_labels[["lumo_energy"]], test_size=0.1, random_state=42
)

history_elumon = elumon.fit(X_pretrain_train, y_pretrain_train, epochs=20, batch_size=32, validation_data=(X_pretrain_val, y_pretrain_val))

Epoch 1/20
1407/1407 [==============================] - 3s 2ms/step - loss: 4.6758 - val_loss: 0.3659
Epoch 2/20
1407/1407 [==============================] - 3s 2ms/step - loss: 0.2713 - val_loss: 0.0984
Epoch 3/20
1407/1407 [==============================] - 3s 2ms/step - loss: 0.1397 - val_loss: 0.0670
Epoch 4/20
1407/1407 [==============================] - 3s 2ms/step - loss: 0.1050 - val_loss: 0.0545
Epoch 5/20
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0834 - val_loss: 0.0427
Epoch 6/20
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0647 - val_loss: 0.0380
Epoch 7/20
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0520 - val_loss: 0.0298
Epoch 8/20
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0420 - val_loss: 0.0236
Epoch 9/20
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0349 - val_loss: 0.0205
Epoch 10/20
1407/1407 [==============================] - 3s 2ms/step - loss: 0.029

In [8]:
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error

In [24]:
X_train_train, X_train_val, y_train_train, y_train_val = train_test_split(train_features[train_features.columns[2:]], train_labels[["homo_lumo_gap"]], test_size=0.02, random_state=42)
X_test = test_features[test_features.columns[2:]].values

encoded_X_train_train = elumon.encoder.predict(X_train_train)
elumo_X_train_train = elumon.predict(X_train_train)
final_X_train_train = np.concatenate((encoded_X_train_train, elumo_X_train_train), axis=1)

encoded_X_train_val = elumon.encoder.predict(X_train_val)
elumo_X_train_val = elumon.predict(X_train_val)
final_X_train_val = np.concatenate((encoded_X_train_val, elumo_X_train_val), axis=1)

encoded_X_test = elumon.encoder.predict(X_test)
elumo_X_test = elumon.predict(X_test)
final_X_test = np.concatenate((encoded_X_test, elumo_X_test), axis=1)

In [25]:
cv = RepeatedKFold(n_splits=3, n_repeats=2, random_state=42)
regressor = LassoCV(cv=cv)

In [26]:
regressor.fit(final_X_train_train, y_train_train.values.ravel())

/work/gallim/anaconda3/envs/ML-projects/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0022410377860069275, tolerance: 0.0007595449569635093
  model = cd_fast.enet_coordinate_descent_gram(
/work/gallim/anaconda3/envs/ML-projects/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.003014160320162773, tolerance: 0.0007595449569635093
  model = cd_fast.enet_coordinate_descent_gram(
/work/gallim/anaconda3/envs/ML-projects/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.002717083552852273, tolerance: 0.0007595449569635093
  model = cd_fast.enet_coordinate_descent_gra

LassoCV(cv=RepeatedKFold(n_repeats=2, n_splits=3, random_state=42))

In [28]:
y_val_pred = regressor.predict(final_X_train_val)

print("Validation MSE:", mean_squared_error(y_train_val.values.ravel(), y_val_pred))

Validation MSE: 0.12019911841500416


In [29]:
y_test_pred = regressor.predict(final_X_test)
output_df = pd.DataFrame({"Id": test_features["Id"], "y": y_test_pred})
output_df.to_csv("submission.csv", index=False)